In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 11,
    "font.serif": ["Computer Modern Roman"],
    "figure.figsize": (15,5)
})

In [2]:
pilatus = pd.read_csv("order111064\order_111064_data.txt", sep=";")


# Aufbereitung der Daten

In [3]:
pilatus['time'] = pd.to_datetime(pilatus['time'], format='%Y%m%d').dt.strftime('%Y.%m.%d')

In [4]:
columnsname_pilatus = ['Station','Datum','Temperatur Tagesmaximum [°C]','Temperatur Tagesminimum [°C]','Temperatur Tagesmittel [°C]','Niederschlag Tagessumme [mm]']
pilatus.columns = columnsname_pilatus

pilatus

,Station,Datum,Temperatur Tagesmaximum [°C],Temperatur Tagesminimum [°C],Temperatur Tagesmittel [°C],Niederschlag Tagessumme [mm]
0,PIL,1981.01.01,-0.9,-11.4,-7.4,11.3
1,PIL,1981.01.02,-4.2,-11.8,-8.9,30.3
2,PIL,1981.01.03,0.5,-7.1,-3.4,40.8
3,PIL,1981.01.04,-1.6,-13.4,-6.2,65.3
4,PIL,1981.01.05,-11.0,-13.2,-12.2,0.5
...,...,...,...,...,...,...
15426,PIL,2023.03.28,1.8,-11.0,-5.1,1.3
15427,PIL,2023.03.29,4.9,-2.5,2.1,0.9
15428,PIL,2023.03.30,6.1,0.2,2.7,1.3
15429,PIL,2023.03.31,2.8,-3.8,-0.7,13.2


## Entfernung der nicht vorhandenen Messungen

In [5]:
pilatus = pilatus.replace('-', np.nan)
pilatus.dropna(inplace=True)
pilatus.isna().sum()

Station                         0
Datum                           0
Temperatur Tagesmaximum [°C]    0
Temperatur Tagesminimum [°C]    0
Temperatur Tagesmittel [°C]     0
Niederschlag Tagessumme [mm]    0
dtype: int64

In [6]:
pilatus['Niederschlag Tagessumme [mm]'] = pilatus['Niederschlag Tagessumme [mm]'].astype('float')

In [7]:
pilatus.dtypes

Station                          object
Datum                            object
Temperatur Tagesmaximum [°C]    float64
Temperatur Tagesminimum [°C]    float64
Temperatur Tagesmittel [°C]     float64
Niederschlag Tagessumme [mm]    float64
dtype: object

## Schneefall

Als Faustregel gilt:

$$Niederschlag * 10 = Schneehöhe$$

Dazu muss die Tagesmitteltemperatur $\leq0$ sein.

In [8]:
pilatus['Schneefall [cm]']  = np.where(pilatus['Temperatur Tagesmittel [°C]']<=0, pilatus['Niederschlag Tagessumme [mm]'],0)

In [9]:
pilatus['Datum'] = pd.to_datetime(pilatus['Datum'], format='%Y.%m.%d')

In [15]:
def plot_schneehoehe(start_date, end_date):
    start_date=pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    mask = (pilatus['Datum'] >= start_date) & (pilatus['Datum'] <= end_date)
    data = pilatus.loc[mask]
    plt.bar(data['Datum'], data['Schneefall [cm]'])
    plt.title(f'Schneefall in [cm] von {str(start_date)[0:10]} bis {str(end_date)[0:10]}')
    plt.xticks(data['Datum'],rotation=45)
    plt.grid()
    plt.show()


interact(plot_schneehoehe,
         start_date = widgets.DatePicker(value=pd.to_datetime('2023-03-01')),
         end_date =widgets.DatePicker(value=pd.to_datetime('2023-04-01')),
         )


interactive(children=(DatePicker(value=Timestamp('2023-03-01 00:00:00'), description='start_date', step=1), Da…

<function __main__.plot_schneehoehe(start_date, end_date)>

c:\Users\Pascal Gitz\anaconda3\envs\engineering\lib\site-packages\traitlets\traitlets.py:714: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  silent = bool(old_value == new_value)
